In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

titanic_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/train.csv"

titanic_df = pd.read_csv(titanic_path)

titanic_df = titanic_df.sample(n=len(titanic_df), random_state=1)

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')
def data_pre_process(keep : list, df : pd.DataFrame, experiment=False):
    new_df = df[keep]
    
    if (experiment == True): 
        for i in new_df.index: 
            new_df.at[i ,"Sex"] = titanic_df[titanic_df["Sex"] == new_df["Sex"].get(i)]["Survived"].mean()
            new_df.at[i ,"Pclass"] = titanic_df[titanic_df["Pclass"] == new_df["Pclass"].get(i)]["Survived"].mean()

    else:
        for i in new_df.index:
            if (new_df["Sex"].get(i) == "female"): 
                new_df.at[i ,"Sex"] = 0
            else: 
                new_df.at[i ,"Sex"] = 1

    
            
    if ("Age" in keep):  
        new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
        new_df['Age'] = scaler.fit_transform(new_df[['Age']])
    if ("Fare" in keep): 
        new_df['Fare'] = imputer.fit_transform(new_df[["Fare"]])
        new_df['Fare'] = scaler.fit_transform(new_df[['Fare']])

    
    return new_df

In [3]:
keep = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]
keep_Experimental = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]
titanic_steady_df = data_pre_process(keep, titanic_df)
titanic_Experimental_df = data_pre_process(keep_Experimental, titanic_df, experiment=True)


C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = scaler.fit_transform(new_df[['Age']])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [4]:
X_steady = titanic_steady_df.drop("Survived", axis=1).values
y_steady = titanic_steady_df["Survived"].values



X_Experimental = titanic_Experimental_df.drop("Survived", axis=1).values
y_Experimental = titanic_Experimental_df["Survived"].values

In [5]:
def cut_block(X : list, y : list, cut=5):
    """
    return X_block, y_block
    """
    n_20 = int(len(X) / cut)
    X_block = []
    y_block = []
    for i in range(cut):
        X_block.append(X[i*n_20:(i+1)*n_20])
        y_block.append(y[i*n_20:(i+1)*n_20])
    return X_block, y_block

def assemble_block(X_block : list, y_block : list, test_i : int, cut=5):
    """
    return:
    x_train, y_train, x_test, y_test
    """
    x_test, y_test = X_block[test_i], y_block[test_i]
    x_train = []
    y_train = []
    for i in range(cut):
        if (i == test_i): continue
        x_train.extend(X_block[i])
        y_train.extend(y_block[i])

    return x_train, y_train, x_test, y_test


In [6]:
def train(x_train : list, y_train : list, max_depth = 5):
    """
    return rf_clf
    """
    # 創建隨機森林分類器實例
    rf_clf = RandomForestClassifier(n_estimators=500, max_depth=max_depth, random_state=31)

    # 訓練分類器
    rf_clf.fit(x_train, y_train)

    

    return rf_clf
def test(x_train : list, y_train : list, x_test : list, y_test : list, rf_clf):
    prediction_train = rf_clf.predict(x_train)
    prediction_test = rf_clf.predict(x_test)
    
    return accuracy_score(prediction_train, y_train), accuracy_score(prediction_test, y_test)


In [14]:
cut = 5

X_steady_block, y_steady_block = cut_block(X_steady, y_steady, cut=cut)
X_Experimental_block, y_Experimental_block = cut_block(X_Experimental, y_Experimental, cut=cut)  



for depth in range(4, 10):
    avg_steady_train = []
    avg_steady_test = []
    avg_Experimental_train = []
    avg_Experimental_test = []
    for i in range(cut):
        x_steady_train, y_steady_train, x_steady_test, y_steady_test = \
            assemble_block(X_steady_block, y_steady_block, i, cut=cut)
        
        x_Experimental_train, y_Experimental_train, x_Experimental_test, y_Experimental_test = \
            assemble_block(X_Experimental_block, y_Experimental_block, i, cut=cut)

        
        rf_clf_steady = train(x_steady_train, y_steady_train, max_depth=depth)
        result = test(x_steady_train, y_steady_train, x_steady_test, y_steady_test, rf_clf_steady)

        avg_steady_train.append(result[0])
        avg_steady_test.append(result[1])


        rf_clf_Experimental = train(x_Experimental_train, y_Experimental_train, max_depth=depth)
        result = test(x_Experimental_train, y_Experimental_train, x_Experimental_test, y_Experimental_test, rf_clf_Experimental)

        avg_Experimental_train.append(result[0])
        avg_Experimental_test.append(result[1])

    print("max_depth = ", depth)

    print(f"avg train:{np.mean(avg_steady_train)}, avg test:{np.mean(avg_steady_test)}")
    print(f"std train:{np.std(avg_steady_train)}, std test:{np.std(avg_steady_test)}")
    print(f"max train:{np.max(avg_steady_train)}, max test:{np.max(avg_steady_test)}")
    print(f"min train:{np.min(avg_steady_train)}, min test:{np.min(avg_steady_test)}")

    print("Experimental")

    print(f"avg train:{np.mean(avg_Experimental_train)}, avg test:{np.mean(avg_Experimental_test)}")
    print(f"std train:{np.std(avg_Experimental_train)}, std test:{np.std(avg_Experimental_test)}")
    print(f"max train:{np.max(avg_Experimental_train)}, max test:{np.max(avg_Experimental_test)}")
    print(f"min train:{np.min(avg_Experimental_train)}, min test:{np.min(avg_Experimental_test)}")

    print()

max_depth =  4
avg train:0.8457865168539325, avg test:0.8258426966292134
std train:0.010784468438456455, std test:0.03908433063129457
max train:0.8651685393258427, max test:0.8876404494382022
min train:0.8328651685393258, min test:0.7696629213483146
Experimental
avg train:0.85, avg test:0.8280898876404494
std train:0.010893662600935598, std test:0.041527457556351274
max train:0.8707865168539326, max test:0.8932584269662921
min train:0.8426966292134831, min test:0.7752808988764045

max_depth =  5
avg train:0.8587078651685394, avg test:0.8269662921348313
std train:0.011361656413571196, std test:0.035424455663897646
max train:0.8764044943820225, max test:0.8820224719101124
min train:0.8426966292134831, min test:0.7808988764044944
Experimental
avg train:0.8612359550561799, avg test:0.8235955056179775
std train:0.011108831423193926, std test:0.03248726358382239
max train:0.8792134831460674, max test:0.8764044943820225
min train:0.8497191011235955, min test:0.7865168539325843

max_depth =  6

In [15]:
keep_normal = ["Survived", "Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]


titanic_normal_df = data_pre_process(keep_normal, titanic_df, experiment=True)


X_normal = titanic_normal_df.drop("Survived", axis=1).values
y_normal = titanic_normal_df["Survived"].values





C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = scaler.fit_transform(new_df[['Age']])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [16]:
final_normal_model = train(X_normal, y_normal, max_depth=5)
test(X_normal, y_normal, X_normal, y_normal, final_normal_model)

(0.8496071829405163, 0.8496071829405163)

In [17]:
test_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/test.csv"


keep_test_normal = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Age"]



titanic_test_df = pd.read_csv(test_path)

titanic_test_normal = data_pre_process(keep_test_normal, titanic_test_df, experiment=True)






C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = imputer.fit_transform(new_df[["Age"]])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = scaler.fit_transform(new_df[['Age']])
C:\Users\weiso131\AppData\Local\Temp\ipykernel_1784\3009682004.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [18]:
all_prediction = final_normal_model.predict(titanic_test_normal.values)


In [19]:
submission_path = "C:/Users/weiso131/Desktop/sklearn/titanic_disaster/titanic/submission.csv"

submission_df = pd.read_csv(submission_path)

for i in range(len(submission_df)):
    index = submission_df.index[i]
    

    submission_df.at[index, "Survived"] = int(all_prediction[i])



In [20]:
print(submission_df)
submission_df.to_csv("submission_weiso.csv", index = False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
